In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cassandra23-ps-2/dataset/sample_submissions.csv
/kaggle/input/cassandra23-ps-2/dataset/train_data.csv
/kaggle/input/cassandra23-ps-2/dataset/test_data.csv


In [5]:
df = pd.read_csv('/kaggle/input/cassandra23-ps-2/dataset/train_data.csv')

In [6]:
tf = pd.read_csv('/kaggle/input/cassandra23-ps-2/dataset/test_data.csv')

In [7]:
hash_df = df.copy()
hash_tf = tf.copy()

In [8]:
idf = df.copy()

In [9]:
itf = tf.copy()

In [10]:
df1 = df.copy()
tf1 = tf.copy()

In [11]:
df2 = df.copy()
tf2 = tf.copy()

In [12]:
df.columns

Index(['UID', 'trip_creation_time', 'route_schedule_uuid', 'route_type',
       'trip_uuid', 'source_center', 'source_name', 'destination_center',
       'destination_name', 'od_start_time', 'od_end_time',
       'start_scan_to_end_scan', 'is_cutoff', 'cutoff_factor',
       'cutoff_timestamp', 'actual_distance_to_destination', 'osrm_time',
       'osrm_distance', 'factor', 'segment_actual_time', 'segment_osrm_time',
       'segment_osrm_distance', 'segment_factor', 'actual_time'],
      dtype='object')

In [13]:
df['trip_uuid'].value_counts()

trip-153689939954743811    56
trip-153846035308581166    56
trip-153741795740530104    56
trip-153750670483274503    55
trip-153819749763881430    54
                           ..
trip-153684758182048957     1
trip-153788048470099142     1
trip-153850313832185205     1
trip-153787939890361711     1
trip-153775443915828230     1
Name: trip_uuid, Length: 13535, dtype: int64

In [14]:
# drop missing values from the column 'route_schedule_uuid'
idf.dropna(subset=['route_schedule_uuid'], inplace=True)


In [15]:
df2.dropna(subset=['route_schedule_uuid'], inplace=True)

In [16]:
idf['route_schedule_uuid'].isna().sum()

0

In [17]:
idf['route_schedule_uuid'].value_counts()

thanos::sroute:4029a8a2-6c74-4b7e-a6d8-f9e069fbcea9    848
thanos::sroute:dca6268f-741a-4d1a-b1b0-aab13095a366    809
thanos::sroute:0456b740-1dad-4929-bbe0-87d8843f5a10    769
thanos::sroute:de5e208e-7641-45e6-8100-4d9fb1e5720d    617
thanos::sroute:a1b25549-1e77-498f-8538-00292e5bd5a2    608
                                                      ... 
thanos::sroute:0a05c445-d943-4f05-82ed-fd90a6d31e87      1
thanos::sroute:a23f7a1b-1d7b-4a98-8843-e421edf4b720      1
thanos::sroute:e13f5311-3045-4759-ab11-93b93352d88b      1
thanos::sroute:e00eb6aa-d792-4b28-81fa-fdee413ef326      1
thanos::sroute:f5939796-baf7-416c-a712-78414a22f2ea      1
Name: route_schedule_uuid, Length: 1469, dtype: int64

In [18]:
from sklearn.preprocessing import LabelEncoder

# initialize label encoder
le = LabelEncoder()

# perform label encoding on trip_schedule_uuid column
idf['route_schedule_uuid'] = le.fit_transform(idf['route_schedule_uuid'])


In [19]:
from sklearn.preprocessing import LabelEncoder

# initialize label encoder
le = LabelEncoder()

# perform label encoding on trip_schedule_uuid column
itf['route_schedule_uuid'] = le.fit_transform(itf['route_schedule_uuid'])

In [20]:
itf.shape[0]

72158

In [21]:
df_ne = df.copy()

In [22]:
tf_ne = tf.copy()

In [23]:
import pandas as pd

# create a sample dataframe

# convert date strings to datetime objects
df_ne['od_start_time'] = pd.to_datetime(df_ne['od_start_time'])
df_ne['od_end_time'] = pd.to_datetime(df_ne['od_end_time'])

# calculate time difference in seconds
idf['time_diff_sec'] = (df_ne['od_end_time'] - df_ne['od_start_time']).dt.total_seconds()

In [24]:
import pandas as pd

# create a sample dataframe

# convert date strings to datetime objects
tf_ne['od_start_time'] = pd.to_datetime(tf_ne['od_start_time'])
tf_ne['od_end_time'] = pd.to_datetime(tf_ne['od_end_time'])

# calculate time difference in seconds
itf['time_diff_sec'] = (tf_ne['od_end_time'] - tf_ne['od_start_time']).dt.total_seconds()

In [25]:
df2['segment_key'] = df['route_schedule_uuid'] + df['source_center'] + df['destination_center']
segment_cols = ['segment_actual_time', 'segment_osrm_distance', 'segment_osrm_time']

for col in segment_cols:
    df2[col + '_sum'] = df2.groupby('segment_key')[col].cumsum()
    
df2 = df2.drop(['route_schedule_uuid','source_center','source_name','destination_center','destination_name','segment_key'], axis=1)

In [26]:
tf2['segment_key'] = tf['route_schedule_uuid'] + tf['source_center'] + tf['destination_center']
segment_cols = ['segment_actual_time', 'segment_osrm_distance', 'segment_osrm_time']

for col in segment_cols:
    tf2[col + '_sum'] = tf2.groupby('segment_key')[col].cumsum()
    
tf2 = tf2.drop(['route_schedule_uuid','source_center','source_name','destination_center','destination_name','segment_key'], axis=1)

In [27]:
tf2.shape

(72158, 21)

In [28]:
df2

,UID,trip_creation_time,route_type,trip_uuid,od_start_time,od_end_time,start_scan_to_end_scan,is_cutoff,cutoff_factor,cutoff_timestamp,...,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,actual_time,segment_actual_time_sum,segment_osrm_distance_sum,segment_osrm_time_sum
0,lWYYrbNEdf,2018-09-15 02:23:18.656742,FTL,trip-153697819865649878,2018-09-15 02:23:18.656742,2018-09-15 06:33:35.824186,713.521495,True,44,2018-09-15 04:45:23,...,1228.735753,2.782826,77.988686,20.793472,28.475783,3.292773,100.0,77.988686,28.475783,20.793472
1,iNJcHQlPCm,2018-09-30 15:07:48.971326,FTL,trip-153832006897106333,2018-09-30 22:38:13.903069,2018-10-01 06:48:37.637477,902.759458,True,22,2018-10-01 05:38:30,...,NaN,3.625248,68.517914,18.787231,34.612259,4.277350,61.0,68.517914,34.612259,18.787231
2,U6N0M43qwR,2018-09-23 22:15:46.635975,FTL,trip-153774094663572416,2018-09-24 03:56:37.016232,2018-09-24 11:15:00.826140,923.666146,True,22,2018-09-24 09:21:30,...,1188.943158,4.415969,70.475386,24.371978,29.867950,5.322072,69.0,NaN,NaN,NaN
3,RrJUneuYNj,2018-09-21 05:11:44.833176,FTL,trip-153750670483274503,2018-09-22 10:08:32.593022,2018-09-22 19:17:22.242888,705.451567,True,176,2018-09-22 14:09:17,...,999.421258,2.946647,101.876844,19.588977,27.794019,4.265799,300.0,101.876844,27.794019,19.588977
4,7QLdD7iRC9,2018-09-18 01:10:49.936747,FTL,trip-153723305402762987,2018-09-18 01:10:49.936747,2018-09-18 17:58:48.449251,1109.682268,True,198,2018-09-18 12:14:05,...,NaN,1.998790,46.571301,22.671495,26.971844,3.553652,286.0,46.571301,26.971844,22.671495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72153,dNuV95njhn,2018-09-29 12:35:30.065739,FTL,trip-153822453006548340,2018-09-29 12:35:30.065739,2018-09-29 21:49:00.323701,1132.865742,True,66,2018-09-29 20:08:31,...,1392.622148,1.950803,55.856356,39.233287,46.999351,1.383605,92.0,3293.413141,2650.118673,2003.924103
72154,ZSrzBfWx77,2018-09-22 11:00:33.571928,Carting,trip-153761403357161860,2018-09-22 11:00:33.571928,2018-09-22 13:46:54.235060,355.951299,True,18,2018-09-22 13:11:21,...,NaN,1.927801,35.976514,13.607794,15.139818,1.456469,30.0,881.353268,446.638421,418.262924
72155,XsIYBRVK2V,2018-09-14 23:01:12.630742,FTL,trip-153696607263047584,2018-09-15 04:20:06.629681,2018-09-15 08:08:11.723169,737.899031,True,44,2018-09-15 06:53:24,...,902.191591,2.148569,44.101033,23.355458,27.510588,2.346207,74.0,1122.238226,596.023123,486.243298
72156,6YwMOShK5E,2018-09-22 06:11:41.998188,FTL,trip-153759670199790692,2018-09-22 06:11:41.998188,2018-09-22 18:10:26.705628,845.738186,True,88,2018-09-22 15:42:27,...,1393.905281,2.206373,37.771823,18.740087,29.929128,2.499566,122.0,527.513633,160.142062,123.972095


In [29]:
idf = pd.get_dummies(idf, columns=['route_type', 'is_cutoff'], drop_first=True)

In [30]:
itf = pd.get_dummies(itf, columns=['route_type', 'is_cutoff'], drop_first=True)

In [31]:
# One-hot encode route_type and is_cutoff columns
df2 = pd.get_dummies(df2, columns=['route_type', 'is_cutoff'], drop_first=True)


In [32]:
df2.columns

Index(['UID', 'trip_creation_time', 'trip_uuid', 'od_start_time',
       'od_end_time', 'start_scan_to_end_scan', 'cutoff_factor',
       'cutoff_timestamp', 'actual_distance_to_destination', 'osrm_time',
       'osrm_distance', 'factor', 'segment_actual_time', 'segment_osrm_time',
       'segment_osrm_distance', 'segment_factor', 'actual_time',
       'segment_actual_time_sum', 'segment_osrm_distance_sum',
       'segment_osrm_time_sum', 'route_type_FTL', 'is_cutoff_True'],
      dtype='object')

In [33]:
df2['is_cutoff_True'].value_counts()

1    56774
0    12489
Name: is_cutoff_True, dtype: int64

In [34]:
tf2 = pd.get_dummies(tf2, columns=['route_type', 'is_cutoff'], drop_first=True)

In [35]:
import pandas as pd

# create a sample dataframe

# convert date strings to datetime objects
df_ne['od_start_time'] = pd.to_datetime(df_ne['od_start_time'])
df_ne['od_end_time'] = pd.to_datetime(df_ne['od_end_time'])

# calculate time difference in seconds
df2['time_diff_sec'] = (df_ne['od_end_time'] - df_ne['od_start_time']).dt.total_seconds()

In [36]:
import pandas as pd

# create a sample dataframe

# convert date strings to datetime objects
df_ne['od_start_time'] = pd.to_datetime(df_ne['od_start_time'])
df_ne['od_end_time'] = pd.to_datetime(df_ne['od_end_time'])

# calculate time difference in seconds
idf['time_diff_sec'] = (df_ne['od_end_time'] - df_ne['od_start_time']).dt.total_seconds()

In [37]:
df2['time_diff_sec']

0         15017.167444
1         29423.734408
2         26303.809908
3         32929.649866
4         60478.512504
             ...      
72153     33210.257962
72154      9980.663132
72155     13685.093488
72156     43124.707440
72157    138033.406494
Name: time_diff_sec, Length: 69263, dtype: float64

In [38]:
tf_ne = tf.copy()

In [39]:
import pandas as pd

# create a sample dataframe

# convert date strings to datetime objects
tf_ne['od_start_time'] = pd.to_datetime(tf_ne['od_start_time'])
tf_ne['od_end_time'] = pd.to_datetime(tf_ne['od_end_time'])

# calculate time difference in seconds
itf['time_diff_sec'] = (tf_ne['od_end_time'] - tf_ne['od_start_time']).dt.total_seconds()

In [40]:
tf2

,UID,trip_creation_time,trip_uuid,od_start_time,od_end_time,start_scan_to_end_scan,cutoff_factor,cutoff_timestamp,actual_distance_to_destination,osrm_time,...,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,segment_actual_time_sum,segment_osrm_distance_sum,segment_osrm_time_sum,route_type_FTL,is_cutoff_True
0,XMsT9clviB,2018-09-20 02:35:36.476840,trip-153741093647649320,2018-09-20 03:21:32.418600,2018-09-20 04:47:45.236797,86.0,27,2018-09-20 04:01:19.505586,27.637279,28.0,...,1.428571,16.0,7.0,10.8152,2.285714,16.0,10.8152,7.0,0,1
1,1iUuyRUZ2Y,2018-09-20 02:35:36.476840,trip-153741093647649320,2018-09-20 03:21:32.418600,2018-09-20 04:47:45.236797,86.0,39,2018-09-20 03:33:55,39.386040,44.0,...,1.545455,6.0,5.0,3.9153,1.200000,22.0,14.7305,12.0,0,0
2,wancwiAIU3,2018-09-20 02:35:36.476840,trip-153741093647649320,2018-09-20 04:47:45.236797,2018-09-20 06:36:55.627764,109.0,9,2018-09-20 06:15:58,10.403038,11.0,...,1.363636,15.0,11.0,12.1171,1.363636,15.0,12.1171,11.0,0,1
3,1tIt8EwJjB,2018-09-20 02:35:36.476840,trip-153741093647649320,2018-09-20 04:47:45.236797,2018-09-20 06:36:55.627764,109.0,36,2018-09-20 05:15:56,38.939167,39.0,...,1.948718,10.0,10.0,11.3648,1.000000,25.0,23.4819,21.0,0,1
4,W2vL6bJrjt,2018-09-23 06:42:06.021680,trip-153768492602129387,2018-09-23 06:42:06.021680,2018-09-23 11:44:28.365845,302.0,22,2018-09-23 11:05:19,23.194334,24.0,...,1.583333,38.0,24.0,26.8622,1.583333,38.0,26.8622,24.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72153,L5IrEpNBFb,2018-09-20 16:24:28.436231,trip-153746066843555182,2018-09-20 16:24:28.436231,2018-09-20 23:32:09.618069,427.0,9,2018-09-20 22:47:17,11.080143,23.0,...,1.913043,44.0,23.0,15.3398,1.913043,1226.0,744.9059,982.0,0,1
72154,s5TDXjnN3s,2018-09-20 16:24:28.436231,trip-153746066843555182,2018-09-20 16:24:28.436231,2018-09-20 23:32:09.618069,427.0,18,2018-09-20 22:35:17,19.221595,35.0,...,1.600000,12.0,26.0,16.9842,0.461538,1238.0,761.8901,1008.0,0,1
72155,0croXy5yK5,2018-09-20 16:24:28.436231,trip-153746066843555182,2018-09-20 16:24:28.436231,2018-09-20 23:32:09.618069,427.0,27,2018-09-20 22:21:19,27.964113,50.0,...,1.400000,13.0,21.0,15.2911,0.619048,1251.0,777.1812,1029.0,0,1
72156,P0okvBa2Ik,2018-09-20 16:24:28.436231,trip-153746066843555182,2018-09-20 16:24:28.436231,2018-09-20 23:32:09.618069,427.0,45,2018-09-20 21:57:20,45.258278,60.0,...,1.566667,12.0,12.0,8.1858,1.000000,1263.0,785.3670,1041.0,0,1


In [41]:
df2['od_start_time'] = pd.to_datetime(df['od_start_time'], errors='coerce')

# check for invalid values



df2['trip_month'] = df2['od_start_time'].dt.month
df2['trip_hour'] = df2['od_start_time'].dt.hour
df2['trip_day'] = df2['od_start_time'].dt.day
df2['trip_minutes'] = df2['od_start_time'].dt.minute
df2['trip_dayofweek'] = df2['od_start_time'].dt.dayofweek

In [42]:
idf['od_start_time'] = pd.to_datetime(idf['od_start_time'], errors='coerce')

# check for invalid values



idf['trip_month'] = idf['od_start_time'].dt.month
idf['trip_hour'] = idf['od_start_time'].dt.hour
idf['trip_day'] = idf['od_start_time'].dt.day
idf['trip_minutes'] = idf['od_start_time'].dt.minute
idf['trip_dayofweek'] = idf['od_start_time'].dt.dayofweek

In [43]:
itf['od_start_time'] = pd.to_datetime(itf['od_start_time'], errors='coerce')

# check for invalid values



itf['trip_month'] = itf['od_start_time'].dt.month
itf['trip_hour'] = itf['od_start_time'].dt.hour
itf['trip_day'] = itf['od_start_time'].dt.day
itf['trip_minutes'] = itf['od_start_time'].dt.minute
itf['trip_dayofweek'] = itf['od_start_time'].dt.dayofweek

In [44]:
df3 = df2.copy()

In [45]:
tf3 = tf2.copy()

In [46]:
df2.dropna(subset=['od_end_time'], inplace=True)

In [47]:
import pandas as pd

# Convert the 'od_end_time' column to datetime format
#df2.dropna(subset=['od_end_time'], inplace=True)

idf['od_end_time'] = pd.to_datetime(idf['od_end_time'])

# Split the datetime into separate columns
idf['trip_month_end'] = idf['od_end_time'].dt.month
idf['trip_minute_end'] = idf['od_end_time'].dt.minute
idf['trip_hour_end'] = idf['od_end_time'].dt.hour
idf['trip_day_end'] =  idf['od_end_time'].dt.day
#trip1['trip_week_end'] = trip1['od_end_time'].dt.isocalendar().week
idf['trip_dayofweek_end'] = idf['od_end_time'].dt.dayofweek



In [48]:
itf['od_end_time'] = pd.to_datetime(itf['od_end_time'])

# Split the datetime into separate columns
itf['trip_month_end'] = itf['od_end_time'].dt.month
itf['trip_minute_end'] = itf['od_end_time'].dt.minute
itf['trip_hour_end'] = itf['od_end_time'].dt.hour
itf['trip_day_end'] =  itf['od_end_time'].dt.day
#trip1['trip_week_end'] = trip1['od_end_time'].dt.isocalendar().week
itf['trip_dayofweek_end'] = itf['od_end_time'].dt.dayofweek


In [49]:
import pandas as pd

# Convert the 'od_end_time' column to datetime format
#df2.dropna(subset=['od_end_time'], inplace=True)

idf['od_end_time'] = pd.to_datetime(tf2['od_end_time'])

# Split the datetime into separate columns
itf['trip_month_end'] = itf['od_end_time'].dt.month
itf['trip_minute_end'] = itf['od_end_time'].dt.minute
itf['trip_hour_end'] = itf['od_end_time'].dt.hour
itf['trip_day_end'] =  itf['od_end_time'].dt.day
#trip1['trip_week_end'] = trip1['od_end_time'].dt.isocalendar().week
itf['trip_dayofweek_end'] = itf['od_end_time'].dt.dayofweek


In [50]:
tf2

,UID,trip_creation_time,trip_uuid,od_start_time,od_end_time,start_scan_to_end_scan,cutoff_factor,cutoff_timestamp,actual_distance_to_destination,osrm_time,...,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,segment_actual_time_sum,segment_osrm_distance_sum,segment_osrm_time_sum,route_type_FTL,is_cutoff_True
0,XMsT9clviB,2018-09-20 02:35:36.476840,trip-153741093647649320,2018-09-20 03:21:32.418600,2018-09-20 04:47:45.236797,86.0,27,2018-09-20 04:01:19.505586,27.637279,28.0,...,1.428571,16.0,7.0,10.8152,2.285714,16.0,10.8152,7.0,0,1
1,1iUuyRUZ2Y,2018-09-20 02:35:36.476840,trip-153741093647649320,2018-09-20 03:21:32.418600,2018-09-20 04:47:45.236797,86.0,39,2018-09-20 03:33:55,39.386040,44.0,...,1.545455,6.0,5.0,3.9153,1.200000,22.0,14.7305,12.0,0,0
2,wancwiAIU3,2018-09-20 02:35:36.476840,trip-153741093647649320,2018-09-20 04:47:45.236797,2018-09-20 06:36:55.627764,109.0,9,2018-09-20 06:15:58,10.403038,11.0,...,1.363636,15.0,11.0,12.1171,1.363636,15.0,12.1171,11.0,0,1
3,1tIt8EwJjB,2018-09-20 02:35:36.476840,trip-153741093647649320,2018-09-20 04:47:45.236797,2018-09-20 06:36:55.627764,109.0,36,2018-09-20 05:15:56,38.939167,39.0,...,1.948718,10.0,10.0,11.3648,1.000000,25.0,23.4819,21.0,0,1
4,W2vL6bJrjt,2018-09-23 06:42:06.021680,trip-153768492602129387,2018-09-23 06:42:06.021680,2018-09-23 11:44:28.365845,302.0,22,2018-09-23 11:05:19,23.194334,24.0,...,1.583333,38.0,24.0,26.8622,1.583333,38.0,26.8622,24.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72153,L5IrEpNBFb,2018-09-20 16:24:28.436231,trip-153746066843555182,2018-09-20 16:24:28.436231,2018-09-20 23:32:09.618069,427.0,9,2018-09-20 22:47:17,11.080143,23.0,...,1.913043,44.0,23.0,15.3398,1.913043,1226.0,744.9059,982.0,0,1
72154,s5TDXjnN3s,2018-09-20 16:24:28.436231,trip-153746066843555182,2018-09-20 16:24:28.436231,2018-09-20 23:32:09.618069,427.0,18,2018-09-20 22:35:17,19.221595,35.0,...,1.600000,12.0,26.0,16.9842,0.461538,1238.0,761.8901,1008.0,0,1
72155,0croXy5yK5,2018-09-20 16:24:28.436231,trip-153746066843555182,2018-09-20 16:24:28.436231,2018-09-20 23:32:09.618069,427.0,27,2018-09-20 22:21:19,27.964113,50.0,...,1.400000,13.0,21.0,15.2911,0.619048,1251.0,777.1812,1029.0,0,1
72156,P0okvBa2Ik,2018-09-20 16:24:28.436231,trip-153746066843555182,2018-09-20 16:24:28.436231,2018-09-20 23:32:09.618069,427.0,45,2018-09-20 21:57:20,45.258278,60.0,...,1.566667,12.0,12.0,8.1858,1.000000,1263.0,785.3670,1041.0,0,1


In [51]:
import pandas as pd

# Convert the 'cutoff_timestamp' column to datetime format
#tf2.dropna(subset=['cutoff_timestamp'], inplace=True)

tf2['cutoff_timestamp'] = pd.to_datetime(tf2['cutoff_timestamp'])

# Split the datetime into separate columns
tf2['cutoff_month'] = tf2['cutoff_timestamp'].dt.month
tf2['cutoff_minute'] = tf2['cutoff_timestamp'].dt.minute
tf2['cutoff_hour'] = tf2['cutoff_timestamp'].dt.hour
tf2['cutoff_day'] = tf2['cutoff_timestamp'].dt.day
#tf2['cutoff_week'] = tf2['cutoff_timestamp'].dt.isocalendar().week
tf2['cutoff_dayofweek'] = tf2['cutoff_timestamp'].dt.dayofweek


In [52]:
import pandas as pd

# Convert the 'cutoff_timestamp' column to datetime format
#tf2.dropna(subset=['cutoff_timestamp'], inplace=True)

itf['cutoff_timestamp'] = pd.to_datetime(itf['cutoff_timestamp'])

# Split the datetime into separate columns
itf['cutoff_month'] = itf['cutoff_timestamp'].dt.month
itf['cutoff_minute'] = itf['cutoff_timestamp'].dt.minute
itf['cutoff_hour'] = itf['cutoff_timestamp'].dt.hour
itf['cutoff_day'] = itf['cutoff_timestamp'].dt.day
#tf2['cutoff_week'] = tf2['cutoff_timestamp'].dt.isocalendar().week
itf['cutoff_dayofweek'] = itf['cutoff_timestamp'].dt.dayofweek

In [53]:
import pandas as pd

# Convert the 'cutoff_timestamp' column to datetime format
df2.dropna(subset=['cutoff_timestamp'], inplace=True)
df2['cutoff_timestamp'] = pd.to_datetime(df2['cutoff_timestamp'])

# Split the datetime into separate columns
df2['cutoff_month'] = df2['cutoff_timestamp'].dt.month
df2['cutoff_minute'] = df2['cutoff_timestamp'].dt.minute
df2['cutoff_hour'] = df2['cutoff_timestamp'].dt.hour
df2['cutoff_day'] =  df2['cutoff_timestamp'].dt.day
df2['cutoff_dayofweek'] = df2['cutoff_timestamp'].dt.dayofweek


In [54]:
idf.dropna(subset=['cutoff_timestamp'], inplace=True)
idf['cutoff_timestamp'] = pd.to_datetime(idf['cutoff_timestamp'])

# Split the datetime into separate columns
idf['cutoff_month'] = idf['cutoff_timestamp'].dt.month
idf['cutoff_minute'] = idf['cutoff_timestamp'].dt.minute
idf['cutoff_hour'] = idf['cutoff_timestamp'].dt.hour
idf['cutoff_day'] =  idf['cutoff_timestamp'].dt.day
idf['cutoff_dayofweek'] = idf['cutoff_timestamp'].dt.dayofweek

In [55]:
df2.shape

(67698, 33)

In [56]:
tf2.shape

(72158, 26)

In [57]:
df2.columns

Index(['UID', 'trip_creation_time', 'trip_uuid', 'od_start_time',
       'od_end_time', 'start_scan_to_end_scan', 'cutoff_factor',
       'cutoff_timestamp', 'actual_distance_to_destination', 'osrm_time',
       'osrm_distance', 'factor', 'segment_actual_time', 'segment_osrm_time',
       'segment_osrm_distance', 'segment_factor', 'actual_time',
       'segment_actual_time_sum', 'segment_osrm_distance_sum',
       'segment_osrm_time_sum', 'route_type_FTL', 'is_cutoff_True',
       'time_diff_sec', 'trip_month', 'trip_hour', 'trip_day', 'trip_minutes',
       'trip_dayofweek', 'cutoff_month', 'cutoff_minute', 'cutoff_hour',
       'cutoff_day', 'cutoff_dayofweek'],
      dtype='object')

In [58]:
df[['osrm_time', 'actual_time']]

,osrm_time,actual_time
0,778.672716,100.0
1,227.479029,61.0
2,400.072069,69.0
3,510.123878,300.0
4,NaN,286.0
...,...,...
72153,NaN,92.0
72154,374.255819,30.0
72155,895.612578,74.0
72156,NaN,122.0


In [59]:
tf[['osrm_distance', 'actual_distance_to_destination']]

,osrm_distance,actual_distance_to_destination
0,32.5395,27.637279
1,54.2181,39.386040
2,12.1171,10.403038
3,47.1900,38.939167
4,26.8622,23.194334
...,...,...
72153,15.3398,11.080143
72154,34.4366,19.221595
72155,50.7273,27.964113
72156,67.9280,45.258278


In [60]:
gf = df2.copy()

In [61]:
df2.shape

(67698, 33)

In [62]:
gft = tf2.copy()

In [63]:
gf = gf.drop(['osrm_time', 'osrm_distance'], axis = 1)

In [64]:
gft = gft.drop(['osrm_time', 'osrm_distance'], axis = 1)

In [65]:
gf.columns

Index(['UID', 'trip_creation_time', 'trip_uuid', 'od_start_time',
       'od_end_time', 'start_scan_to_end_scan', 'cutoff_factor',
       'cutoff_timestamp', 'actual_distance_to_destination', 'factor',
       'segment_actual_time', 'segment_osrm_time', 'segment_osrm_distance',
       'segment_factor', 'actual_time', 'segment_actual_time_sum',
       'segment_osrm_distance_sum', 'segment_osrm_time_sum', 'route_type_FTL',
       'is_cutoff_True', 'time_diff_sec', 'trip_month', 'trip_hour',
       'trip_day', 'trip_minutes', 'trip_dayofweek', 'cutoff_month',
       'cutoff_minute', 'cutoff_hour', 'cutoff_day', 'cutoff_dayofweek'],
      dtype='object')

In [66]:
gf = gf.drop(['UID', 'trip_creation_time', 'trip_uuid', 'od_start_time',
       'od_end_time',
       'cutoff_timestamp'], axis = 1)

In [67]:
gft = gft.drop(['UID', 'trip_creation_time', 'trip_uuid', 'od_start_time',
       'od_end_time',
       'cutoff_timestamp'], axis = 1)

In [68]:
gf


,start_scan_to_end_scan,cutoff_factor,actual_distance_to_destination,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,actual_time,segment_actual_time_sum,...,trip_month,trip_hour,trip_day,trip_minutes,trip_dayofweek,cutoff_month,cutoff_minute,cutoff_hour,cutoff_day,cutoff_dayofweek
0,713.521495,44,158.807515,2.782826,77.988686,20.793472,28.475783,3.292773,100.0,77.988686,...,9,2,15,23,5,9,45,4,15,5
1,902.759458,22,81.388109,3.625248,68.517914,18.787231,34.612259,4.277350,61.0,68.517914,...,9,22,30,38,6,10,38,5,1,0
2,923.666146,22,134.486898,4.415969,70.475386,24.371978,29.867950,5.322072,69.0,NaN,...,9,3,24,56,0,9,21,9,24,0
3,705.451567,176,262.866674,2.946647,101.876844,19.588977,27.794019,4.265799,300.0,101.876844,...,9,10,22,8,5,9,9,14,22,5
4,1109.682268,198,379.389280,1.998790,46.571301,22.671495,26.971844,3.553652,286.0,46.571301,...,9,1,18,10,1,9,14,12,18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72153,1132.865742,66,135.001150,1.950803,55.856356,39.233287,46.999351,1.383605,92.0,3293.413141,...,9,12,29,35,5,9,8,20,29,5
72154,355.951299,18,74.245265,1.927801,35.976514,13.607794,15.139818,1.456469,30.0,881.353268,...,9,11,22,0,5,9,11,13,22,5
72155,737.899031,44,54.097087,2.148569,44.101033,23.355458,27.510588,2.346207,74.0,1122.238226,...,9,4,15,20,5,9,53,6,15,5
72156,845.738186,88,99.918631,2.206373,37.771823,18.740087,29.929128,2.499566,122.0,527.513633,...,9,6,22,11,5,9,42,15,22,5


In [69]:
gft.shape

(72158, 18)

In [70]:
gf.shape

(67698, 25)

In [71]:
gf

,start_scan_to_end_scan,cutoff_factor,actual_distance_to_destination,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,actual_time,segment_actual_time_sum,...,trip_month,trip_hour,trip_day,trip_minutes,trip_dayofweek,cutoff_month,cutoff_minute,cutoff_hour,cutoff_day,cutoff_dayofweek
0,713.521495,44,158.807515,2.782826,77.988686,20.793472,28.475783,3.292773,100.0,77.988686,...,9,2,15,23,5,9,45,4,15,5
1,902.759458,22,81.388109,3.625248,68.517914,18.787231,34.612259,4.277350,61.0,68.517914,...,9,22,30,38,6,10,38,5,1,0
2,923.666146,22,134.486898,4.415969,70.475386,24.371978,29.867950,5.322072,69.0,NaN,...,9,3,24,56,0,9,21,9,24,0
3,705.451567,176,262.866674,2.946647,101.876844,19.588977,27.794019,4.265799,300.0,101.876844,...,9,10,22,8,5,9,9,14,22,5
4,1109.682268,198,379.389280,1.998790,46.571301,22.671495,26.971844,3.553652,286.0,46.571301,...,9,1,18,10,1,9,14,12,18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72153,1132.865742,66,135.001150,1.950803,55.856356,39.233287,46.999351,1.383605,92.0,3293.413141,...,9,12,29,35,5,9,8,20,29,5
72154,355.951299,18,74.245265,1.927801,35.976514,13.607794,15.139818,1.456469,30.0,881.353268,...,9,11,22,0,5,9,11,13,22,5
72155,737.899031,44,54.097087,2.148569,44.101033,23.355458,27.510588,2.346207,74.0,1122.238226,...,9,4,15,20,5,9,53,6,15,5
72156,845.738186,88,99.918631,2.206373,37.771823,18.740087,29.929128,2.499566,122.0,527.513633,...,9,6,22,11,5,9,42,15,22,5


In [72]:
gf.corr()

,start_scan_to_end_scan,cutoff_factor,actual_distance_to_destination,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,actual_time,segment_actual_time_sum,...,trip_month,trip_hour,trip_day,trip_minutes,trip_dayofweek,cutoff_month,cutoff_minute,cutoff_hour,cutoff_day,cutoff_dayofweek
start_scan_to_end_scan,1.000000,0.773103,0.765098,-0.022155,0.090977,0.207390,0.284452,-0.023844,0.774840,0.701931,...,0.007889,0.115446,0.013400,-0.008330,0.008024,0.053114,0.000843,0.140620,0.000348,0.021059
cutoff_factor,0.773103,1.000000,0.989359,-0.063458,0.047349,0.151870,0.220034,-0.034155,0.979123,0.573338,...,0.011911,0.076817,0.004406,-0.012905,0.004434,0.027949,0.003398,0.099306,0.001350,0.009645
actual_distance_to_destination,0.765098,0.989359,1.000000,-0.062844,0.046956,0.151344,0.218455,-0.033546,0.968630,0.567445,...,0.011509,0.075729,0.004444,-0.012369,0.004592,0.027464,0.004190,0.098043,0.001345,0.009728
factor,-0.022155,-0.063458,-0.062844,1.000000,0.480324,-0.049434,-0.034041,0.504951,0.032209,-0.039209,...,-0.002881,-0.048903,0.010363,-0.009487,-0.000488,-0.006213,-0.001693,-0.035305,0.010057,0.000323
segment_actual_time,0.090977,0.047349,0.046956,0.480324,1.000000,0.454025,0.470648,0.461484,0.121462,0.046341,...,-0.006235,-0.011961,0.008416,-0.007869,0.008135,-0.004538,-0.000706,0.001938,0.007519,0.007725
segment_osrm_time,0.207390,0.151870,0.151344,-0.049434,0.454025,1.000000,0.918812,-0.070559,0.164659,0.141626,...,-0.004738,0.023781,0.004047,0.001881,0.004690,0.003985,0.000622,0.050732,0.002097,0.004062
segment_osrm_distance,0.284452,0.220034,0.218455,-0.034041,0.470648,0.918812,1.000000,-0.059873,0.230284,0.195149,...,-0.004004,0.035211,0.005822,-0.001905,0.006486,0.007198,0.000983,0.062077,0.004130,0.008427
segment_factor,-0.023844,-0.034155,-0.033546,0.504951,0.461484,-0.070559,-0.059873,1.000000,0.012625,-0.028086,...,-0.001852,-0.025718,0.004097,-0.005337,-0.000306,-0.004671,0.004039,-0.016731,0.004382,-0.000091
actual_time,0.774840,0.979123,0.968630,0.032209,0.121462,0.164659,0.230284,0.012625,1.000000,0.557602,...,0.009299,0.081329,0.006957,-0.013044,0.003070,0.024493,0.002803,0.090719,0.004564,0.013000
segment_actual_time_sum,0.701931,0.573338,0.567445,-0.039209,0.046341,0.141626,0.195149,-0.028086,0.557602,1.000000,...,-0.002840,0.066688,0.009448,-0.003751,0.006658,0.025357,0.000463,0.091252,0.006443,0.010507


In [73]:
gf = gf.interpolate(method ='linear', limit_direction ='forward')

In [74]:
null_cols = gf.isnull().any()

In [75]:
print(null_cols)

start_scan_to_end_scan            False
cutoff_factor                     False
actual_distance_to_destination    False
factor                            False
segment_actual_time               False
segment_osrm_time                 False
segment_osrm_distance             False
segment_factor                    False
actual_time                       False
segment_actual_time_sum           False
segment_osrm_distance_sum         False
segment_osrm_time_sum             False
route_type_FTL                    False
is_cutoff_True                    False
time_diff_sec                     False
trip_month                        False
trip_hour                         False
trip_day                          False
trip_minutes                      False
trip_dayofweek                    False
cutoff_month                      False
cutoff_minute                     False
cutoff_hour                       False
cutoff_day                        False
cutoff_dayofweek                  False


In [76]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Separate the target variable and the features
target_col = 'actual_time'
X = gf.drop(columns=[target_col])
y = gf[target_col]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost regressor
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators = 1500, seed = 42)

# Train the model on the training set
xg_reg.fit(X_train,y_train)

# Make predictions on the test set
preds = xg_reg.predict(X_test)

# Calculate the RMSE value
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE value:", rmse)

RMSE value: 45.49766934612937


In [77]:
idf.shape

(68378, 40)

In [78]:
itf.shape

(72158, 39)

In [79]:
idf.columns

Index(['UID', 'trip_creation_time', 'route_schedule_uuid', 'trip_uuid',
       'source_center', 'source_name', 'destination_center',
       'destination_name', 'od_start_time', 'od_end_time',
       'start_scan_to_end_scan', 'cutoff_factor', 'cutoff_timestamp',
       'actual_distance_to_destination', 'osrm_time', 'osrm_distance',
       'factor', 'segment_actual_time', 'segment_osrm_time',
       'segment_osrm_distance', 'segment_factor', 'actual_time',
       'time_diff_sec', 'route_type_FTL', 'is_cutoff_True', 'trip_month',
       'trip_hour', 'trip_day', 'trip_minutes', 'trip_dayofweek',
       'trip_month_end', 'trip_minute_end', 'trip_hour_end', 'trip_day_end',
       'trip_dayofweek_end', 'cutoff_month', 'cutoff_minute', 'cutoff_hour',
       'cutoff_day', 'cutoff_dayofweek'],
      dtype='object')

In [80]:
import hashlib

# define the hash function
def hash_encode(column):
    encoded = column.apply(lambda x: int(hashlib.sha256(x.encode('utf-8')).hexdigest(), 16) % 10**8)
    return encoded

# apply the hash function to the 'source_center' and 'destination_name' columns
idf['source_center_hash'] = hash_encode(idf['source_center'])
idf['source_name_hash'] = hash_encode(idf['source_name'])
idf['destination_name_hash'] = hash_encode(idf['destination_name'])

# drop the original 'source_center' and 'destination_name' columns
idf = idf.drop(['source_center', 'destination_name', 'source_name'], axis=1)


In [81]:
import hashlib

# define the hash function
def hash_encode(column):
    encoded = column.apply(lambda x: int(hashlib.sha256(x.encode('utf-8')).hexdigest(), 16) % 10**8)
    return encoded

# apply the hash function to the 'source_center' and 'destination_name' columns
itf['source_center_hash'] = hash_encode(itf['source_center'])
itf['source_name_hash'] = hash_encode(itf['source_name'])
itf['destination_name_hash'] = hash_encode(itf['destination_name'])

# drop the original 'source_center' and 'destination_name' columns
itf = itf.drop(['source_center', 'destination_name', 'source_name'], axis=1)


In [82]:
idf.columns

Index(['UID', 'trip_creation_time', 'route_schedule_uuid', 'trip_uuid',
       'destination_center', 'od_start_time', 'od_end_time',
       'start_scan_to_end_scan', 'cutoff_factor', 'cutoff_timestamp',
       'actual_distance_to_destination', 'osrm_time', 'osrm_distance',
       'factor', 'segment_actual_time', 'segment_osrm_time',
       'segment_osrm_distance', 'segment_factor', 'actual_time',
       'time_diff_sec', 'route_type_FTL', 'is_cutoff_True', 'trip_month',
       'trip_hour', 'trip_day', 'trip_minutes', 'trip_dayofweek',
       'trip_month_end', 'trip_minute_end', 'trip_hour_end', 'trip_day_end',
       'trip_dayofweek_end', 'cutoff_month', 'cutoff_minute', 'cutoff_hour',
       'cutoff_day', 'cutoff_dayofweek', 'source_center_hash',
       'source_name_hash', 'destination_name_hash'],
      dtype='object')

In [83]:
idf = idf.drop(['UID', 'trip_creation_time', 'trip_uuid',
       'destination_center', 'od_start_time', 'od_end_time', 'cutoff_timestamp'
       ], axis = 1)

In [84]:
itf = itf.drop(['UID', 'trip_creation_time', 'trip_uuid',
       'destination_center', 'od_start_time', 'od_end_time', 'cutoff_timestamp'
       ], axis = 1)

In [85]:
idf.isnull().any()

route_schedule_uuid               False
start_scan_to_end_scan            False
cutoff_factor                     False
actual_distance_to_destination    False
osrm_time                          True
osrm_distance                      True
factor                             True
segment_actual_time               False
segment_osrm_time                 False
segment_osrm_distance             False
segment_factor                     True
actual_time                       False
time_diff_sec                      True
route_type_FTL                    False
is_cutoff_True                    False
trip_month                        False
trip_hour                         False
trip_day                          False
trip_minutes                      False
trip_dayofweek                    False
trip_month_end                     True
trip_minute_end                    True
trip_hour_end                      True
trip_day_end                       True
trip_dayofweek_end                 True


In [86]:
df['osrm_time'].corr(df['cutoff_factor'])

0.7357573149390013

In [87]:
idf2 = idf.copy()

In [88]:
itf2 = itf.copy()

In [89]:
idf = idf.drop(['osrm_time', 'osrm_distance'], axis = 1)

In [90]:
itf = itf.drop(['osrm_time', 'osrm_distance'], axis = 1)

In [91]:
idf.isnull().any()

route_schedule_uuid               False
start_scan_to_end_scan            False
cutoff_factor                     False
actual_distance_to_destination    False
factor                             True
segment_actual_time               False
segment_osrm_time                 False
segment_osrm_distance             False
segment_factor                     True
actual_time                       False
time_diff_sec                      True
route_type_FTL                    False
is_cutoff_True                    False
trip_month                        False
trip_hour                         False
trip_day                          False
trip_minutes                      False
trip_dayofweek                    False
trip_month_end                     True
trip_minute_end                    True
trip_hour_end                      True
trip_day_end                       True
trip_dayofweek_end                 True
cutoff_month                      False
cutoff_minute                     False


In [92]:
idf.dtypes

route_schedule_uuid                 int64
start_scan_to_end_scan            float64
cutoff_factor                       int64
actual_distance_to_destination    float64
factor                            float64
segment_actual_time               float64
segment_osrm_time                 float64
segment_osrm_distance             float64
segment_factor                    float64
actual_time                       float64
time_diff_sec                     float64
route_type_FTL                      uint8
is_cutoff_True                      uint8
trip_month                          int64
trip_hour                           int64
trip_day                            int64
trip_minutes                        int64
trip_dayofweek                      int64
trip_month_end                    float64
trip_minute_end                   float64
trip_hour_end                     float64
trip_day_end                      float64
trip_dayofweek_end                float64
cutoff_month                      

In [93]:
icf = idf.copy()

In [94]:
ikf = idf.copy()

In [95]:
icf.dropna(subset=['trip_month_end', 'trip_minute_end', 'trip_hour_end', 'trip_day_end', 'trip_dayofweek_end'], inplace=True)


In [96]:
icf.shape

(67698, 31)

In [97]:
idf.shape

(68378, 31)

In [98]:
idf.columns

Index(['route_schedule_uuid', 'start_scan_to_end_scan', 'cutoff_factor',
       'actual_distance_to_destination', 'factor', 'segment_actual_time',
       'segment_osrm_time', 'segment_osrm_distance', 'segment_factor',
       'actual_time', 'time_diff_sec', 'route_type_FTL', 'is_cutoff_True',
       'trip_month', 'trip_hour', 'trip_day', 'trip_minutes', 'trip_dayofweek',
       'trip_month_end', 'trip_minute_end', 'trip_hour_end', 'trip_day_end',
       'trip_dayofweek_end', 'cutoff_month', 'cutoff_minute', 'cutoff_hour',
       'cutoff_day', 'cutoff_dayofweek', 'source_center_hash',
       'source_name_hash', 'destination_name_hash'],
      dtype='object')

In [99]:
icf.isnull().any()

route_schedule_uuid               False
start_scan_to_end_scan            False
cutoff_factor                     False
actual_distance_to_destination    False
factor                             True
segment_actual_time               False
segment_osrm_time                 False
segment_osrm_distance             False
segment_factor                     True
actual_time                       False
time_diff_sec                     False
route_type_FTL                    False
is_cutoff_True                    False
trip_month                        False
trip_hour                         False
trip_day                          False
trip_minutes                      False
trip_dayofweek                    False
trip_month_end                    False
trip_minute_end                   False
trip_hour_end                     False
trip_day_end                      False
trip_dayofweek_end                False
cutoff_month                      False
cutoff_minute                     False


In [100]:
# create a new dataframe for regression analysis
reg_df = icf.copy()

# remove the 'factor' and 'segment_factor' columns
reg_df = reg_df.drop(['factor', 'segment_factor'], axis=1)

# remove rows with any missing values
reg_df = reg_df.dropna()

# separate the independent variables and dependent variable
X = icf.drop('factor', axis=1)
y = icf['factor']

# split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # instantiate and fit the linear regression model
# lin_reg = LinearRegression()
# lin_reg.fit(X_train, y_train)

# # predict the missing factor values
# factor_pred = lin_reg.predict(icf[X.columns][icf['factor'].isna()])

# # fill in the missing factor values
# icf.loc[icf['factor'].isna(), 'factor'] = factor_pred

# # repeat the same process for 'segment_factor'
# reg_df = icf.copy()
# reg_df = reg_df.drop(['factor', 'segment_factor'], axis=1)
# reg_df = reg_df.dropna()

# X = reg_df.drop('segment_factor', axis=1)
# y = reg_df['segment_factor']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# lin_reg = LinearRegression()
# lin_reg.fit(X_train, y_train)

# segment_factor_pred = lin_reg.predict(icf[X.columns][icf['segment_factor'].isna()])

# icf.loc[icf['segment_factor'].isna(), 'segment_factor'] = segment_factor_pred


In [101]:
# create a new dataframe for regression analysis
from sklearn.linear_model import LinearRegression
reg_df = icf.dropna().copy()

# separate the independent variables and dependent variable
X = reg_df.drop(['factor', 'segment_factor'], axis=1)
y_factor = reg_df['factor']
y_segment_factor = reg_df['segment_factor']

# split the data into training and testing sets
X_train_factor, X_test_factor, y_train_factor, y_test_factor = train_test_split(X, y_factor, test_size=0.3, random_state=42)
X_train_segment_factor, X_test_segment_factor, y_train_segment_factor, y_test_segment_factor = train_test_split(X, y_segment_factor, test_size=0.3, random_state=42)

# instantiate and fit the linear regression model
lin_reg_factor = LinearRegression()
lin_reg_factor.fit(X_train_factor, y_train_factor)

lin_reg_segment_factor = LinearRegression()
lin_reg_segment_factor.fit(X_train_segment_factor, y_train_segment_factor)

# predict the missing factor values
factor_pred = lin_reg_factor.predict(icf[X.columns][icf['factor'].isna()])
segment_factor_pred = lin_reg_segment_factor.predict(icf[X.columns][icf['segment_factor'].isna()])

# fill in the missing factor values
icf.loc[icf['factor'].isna(), 'factor'] = factor_pred
icf.loc[icf['segment_factor'].isna(), 'segment_factor'] = segment_factor_pred


In [102]:
icf.isnull().any()

route_schedule_uuid               False
start_scan_to_end_scan            False
cutoff_factor                     False
actual_distance_to_destination    False
factor                            False
segment_actual_time               False
segment_osrm_time                 False
segment_osrm_distance             False
segment_factor                    False
actual_time                       False
time_diff_sec                     False
route_type_FTL                    False
is_cutoff_True                    False
trip_month                        False
trip_hour                         False
trip_day                          False
trip_minutes                      False
trip_dayofweek                    False
trip_month_end                    False
trip_minute_end                   False
trip_hour_end                     False
trip_day_end                      False
trip_dayofweek_end                False
cutoff_month                      False
cutoff_minute                     False


In [103]:
# import xgboost as xgb
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split

# # Separate the target variable and the features
# target_col = 'actual_time'
# X = icf.drop(columns=[target_col])
# y = icf[target_col]

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initialize the XGBoost regressor
# xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators = 1500, seed = 42)

# # Train the model on the training set
# xg_reg.fit(X_train,y_train)

# # Make predictions on the test set
# preds = xg_reg.predict(X_test)

# # Calculate the RMSE value
# rmse = np.sqrt(mean_squared_error(y_test, preds))
# print("RMSE value:", rmse)

In [104]:
# gt_3 = tf['UID']
# gt_9 = xg_reg.predict(itf)
# gt_9 = pd.DataFrame(gt_9)
# gt_9.rename(columns={0: "actual_time"}, inplace=True)
# submission_n = pd.concat([gt_3, gt_9], axis=1)
# submission_n.reset_index(drop=True)
# submission_n.to_csv('submission_n.csv', index=False)

In [105]:
# submission_n

In [106]:
pf = df.copy()

In [107]:
tpf = tf.copy()

In [108]:
df.dtypes

UID                                object
trip_creation_time                 object
route_schedule_uuid                object
route_type                         object
trip_uuid                          object
source_center                      object
source_name                        object
destination_center                 object
destination_name                   object
od_start_time                      object
od_end_time                        object
start_scan_to_end_scan            float64
is_cutoff                            bool
cutoff_factor                       int64
cutoff_timestamp                   object
actual_distance_to_destination    float64
osrm_time                         float64
osrm_distance                     float64
factor                            float64
segment_actual_time               float64
segment_osrm_time                 float64
segment_osrm_distance             float64
segment_factor                    float64
actual_time                       

In [109]:
import hashlib

# define a function to hash the uuid column
def hash_uuid(uuid):
    return int(hashlib.sha256(uuid.encode('utf-8')).hexdigest(), 16) % 10**8
pf['route_schedule_uuid'] = pf['route_schedule_uuid'].astype(str)

# hash the route_schedule_uuid column in df
icf['route_schedule_uuid_hash'] = pf['route_schedule_uuid'].apply(hash_uuid)

# concatenate the hashed column to icf
#icf = pd.concat([icf, icf['route_schedule_uuid_hash']], axis=1)


In [110]:
import hashlib

# define a function to hash the uuid column
def hash_uuid(uuid):
    return int(hashlib.sha256(uuid.encode('utf-8')).hexdigest(), 16) % 10**8
tpf['route_schedule_uuid'] = tpf['route_schedule_uuid'].astype(str)

# hash the route_schedule_uuid column in df
itf['route_schedule_uuid_hash'] = pf['route_schedule_uuid'].apply(hash_uuid)

In [111]:
icf = icf.drop(['route_schedule_uuid'], axis = 1)

In [112]:
itf = itf.drop(['route_schedule_uuid'], axis = 1)

In [113]:
icf.isnull().any()

start_scan_to_end_scan            False
cutoff_factor                     False
actual_distance_to_destination    False
factor                            False
segment_actual_time               False
segment_osrm_time                 False
segment_osrm_distance             False
segment_factor                    False
actual_time                       False
time_diff_sec                     False
route_type_FTL                    False
is_cutoff_True                    False
trip_month                        False
trip_hour                         False
trip_day                          False
trip_minutes                      False
trip_dayofweek                    False
trip_month_end                    False
trip_minute_end                   False
trip_hour_end                     False
trip_day_end                      False
trip_dayofweek_end                False
cutoff_month                      False
cutoff_minute                     False
cutoff_hour                       False


In [114]:
icf = icf.drop(['route_schedule_uuid_hash'], axis = 1)

In [115]:
icf.isnull().any()

start_scan_to_end_scan            False
cutoff_factor                     False
actual_distance_to_destination    False
factor                            False
segment_actual_time               False
segment_osrm_time                 False
segment_osrm_distance             False
segment_factor                    False
actual_time                       False
time_diff_sec                     False
route_type_FTL                    False
is_cutoff_True                    False
trip_month                        False
trip_hour                         False
trip_day                          False
trip_minutes                      False
trip_dayofweek                    False
trip_month_end                    False
trip_minute_end                   False
trip_hour_end                     False
trip_day_end                      False
trip_dayofweek_end                False
cutoff_month                      False
cutoff_minute                     False
cutoff_hour                       False


In [116]:
itf.isnull().any()

start_scan_to_end_scan            False
cutoff_factor                     False
actual_distance_to_destination    False
factor                            False
segment_actual_time               False
segment_osrm_time                 False
segment_osrm_distance             False
segment_factor                    False
time_diff_sec                     False
route_type_FTL                    False
is_cutoff_True                    False
trip_month                        False
trip_hour                         False
trip_day                          False
trip_minutes                      False
trip_dayofweek                    False
trip_month_end                    False
trip_minute_end                   False
trip_hour_end                     False
trip_day_end                      False
trip_dayofweek_end                False
cutoff_month                      False
cutoff_minute                     False
cutoff_hour                       False
cutoff_day                        False


In [117]:
# import xgboost as xgb
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split

# # Separate the target variable and the features
# target_col = 'actual_time'
# X = icf.drop(columns=[target_col])
# y = icf[target_col]

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initialize the XGBoost regressor
# xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators =1500, seed = 42)

# # Train the model on the training set
# xg_reg.fit(X_train,y_train)

# # Make predictions on the test set
# preds = xg_reg.predict(X_test)

# # Calculate the RMSE value
# rmse = np.sqrt(mean_squared_error(y_test, preds))
# print("RMSE value:", rmse)

RMSE value: 35.448930591688715


In [118]:
# import pandas as pd
# import xgboost as xgb
# from sklearn.feature_selection import RFE

# # Load the dataset
# #df = pd.read_csv('your_dataset.csv')

# # Split the data into independent variables (X) and dependent variable (y)
# X = icf.drop(['actual_time'], axis=1)
# y = icf['actual_time']

# # Create an XGBoost regressor model
# model = xgb.XGBRegressor()

# # Use RFE to select top 5 features
# rfe = RFE(model, n_features_to_select=15)
# fit = rfe.fit(X, y)

# # Print the top 5 features
# print("Top 5 Features:")
# for i in range(X.shape[1]):
#     if fit.support_[i]:
#         print(X.columns[i])


In [119]:
# import xgboost as xgb
# from xgboost import plot_importance

# # define X and y variables
# X = icf.drop(['actual_time'], axis=1)
# y = icf['actual_time']

# # split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # define XGBoost model
# xgb_model = xgb.XGBRegressor()

# # fit the model
# xgb_model.fit(X_train, y_train)

# # calculate feature importance
# feat_imp = pd.Series(xgb_model.feature_importances_, index=X_train.columns)

# # sort the features by importance in descending order
# feat_imp = feat_imp.sort_values(ascending=False)

# # print the sorted feature importance
# print(feat_imp)

# # plot the feature importance
# plot_importance(xgb_model, max_num_features=14)


In [120]:
# from sklearn.linear_model import Lasso, Ridge

# # Separate the independent variables and dependent variable
# X = icf.drop('actual_time', axis=1)
# y = icf['actual_time']

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # Instantiate and fit the Lasso Regression model
# lasso_reg = Lasso(alpha=0.1)
# lasso_reg.fit(X_train, y_train)

# # Print the coefficients of the Lasso Regression model
# print("Lasso Regression Coefficients:")
# for feature, coef in zip(X.columns, lasso_reg.coef_):
#     print(feature, ':', coef)

# # Instantiate and fit the Ridge Regression model
# ridge_reg = Ridge(alpha=0.1)
# ridge_reg.fit(X_train, y_train)

# # Print the coefficients of the Ridge Regression model
# print("Ridge Regression Coefficients:")
# for feature, coef in zip(X.columns, ridge_reg.coef_):
#     print(feature, ':', coef)


In [121]:
ixf = icf.copy()

In [122]:
ixf = ixf.drop('cutoff_month', axis = 1)

In [123]:
# import xgboost as xgb
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split

# # Separate the target variable and the features
# target_col = 'actual_time'
# X = ixf.drop(columns=[target_col])
# y = ixf[target_col]

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initialize the XGBoost regressor
# xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators =1500, seed = 42)

# # Train the model on the training set
# xg_reg.fit(X_train,y_train)

# # Make predictions on the test set
# preds = xg_reg.predict(X_test)

# # Calculate the RMSE value
# rmse = np.sqrt(mean_squared_error(y_test, preds))
# print("RMSE value:", rmse)

In [124]:
# from sklearn.linear_model import BayesianRidge

# # Define the model
# target_col = 'actual_time'
# X = icf.drop(columns=[target_col])
# y = icf[target_col]

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# model = BayesianRidge()

# # Fit the model on the training data
# model.fit(X_train, y_train)

# # Make predictions on the test data
# y_pred = model.predict(X_test)

# # Calculate the RMSE
# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print("RMSE:", rmse)


In [125]:
# from sklearn.svm import SVR
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error

# # Split the data into training and testing sets
# target_col = 'actual_time'
# X = icf.drop(columns=[target_col])
# y = icf[target_col]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train the SVR model
# svr = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
# svr.fit(X_train, y_train)

# # Predict on the test set
# y_pred = svr.predict(X_test)

# # Calculate the root mean squared error
# rmse = mean_squared_error(y_test, y_pred, squared=False)
# print("SVR RMSE:", rmse)


In [126]:
# from catboost import CatBoostRegressor

# # define the model
# cat_model = CatBoostRegressor(iterations=10000, learning_rate=0.1, loss_function='RMSE')
# target_col = 'actual_time'
# X = icf.drop(columns=[target_col])
# y = icf[target_col]
# # fit the model
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=100)

# # make predictions on the test set
# y_pred = cat_model.predict(X_test)

# # calculate RMSE
# rmse = mean_squared_error(y_test, y_pred, squared=False)
# print('RMSE:', rmse)




In [127]:
# gt_3 = tf['UID']
# gt_9 = cat_model.predict(itf)
# gt_9 = pd.DataFrame(gt_9)
# gt_9.rename(columns={0: "actual_time"}, inplace=True)
# submission_z = pd.concat([gt_3, gt_9], axis=1)
# submission_z.reset_index(drop=True)
# submission_z.to_csv('submission_z.csv', index=False)

In [128]:
#submission_z

In [129]:
# import lightgbm as lgb
# from sklearn.metrics import mean_squared_error

# # Split data into train and validation sets
# X = icf.drop(columns=[target_col])
# y = icf[target_col]
# # fit the model
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
# train_data = lgb.Dataset(X_train, label=y_train)
# valid_data = lgb.Dataset(X_valid, label=y_valid)

# # Set hyperparameters
# params = {
#     "objective": "regression",
#     "metric": "rmse",
#     "boosting_type": "gbdt",
#     "num_leaves": 150,
#     "learning_rate": 0.05,
#     "feature_fraction": 0.9,
#     "bagging_fraction": 0.8,
#     "bagging_freq": 5,
#     "verbose": -1,
# }

# # Train the model
# num_rounds = 5000
# model = lgb.train(
#     params, 
#     train_data, 
#     num_rounds, 
#     valid_sets=[train_data, valid_data], 
#     early_stopping_rounds=100, 
#     verbose_eval=50,
# )

# # Predict on test data
# y_pred = model.predict(X_test)

# # Calculate RMSE
# rmse = mean_squared_error(y_test, y_pred, squared=False)
# print("RMSE:", rmse)


In [130]:
# gt_3 = tf['UID']
# gt_9 = model.predict(itf)
# gt_9 = pd.DataFrame(gt_9)
# gt_9.rename(columns={0: "actual_time"}, inplace=True)
# submission_z = pd.concat([gt_3, gt_9], axis=1)
# submission_z.reset_index(drop=True)
# submission_z.to_csv('submission_z.csv', index=False)

In [131]:
#pip install ngboost


In [132]:
# from ngboost import NGBoost
# from ngboost.learners import default_tree_learner
# from ngboost.scores import MLE, CRPS
# from ngboost.distns import Normal
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error

# # split data into train and test sets
# X = icf.drop(columns=[target_col])
# y = icf[target_col]
# # fit the model
# #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # define the NGBoost model
# ngb = NGBoost(Base=default_tree_learner, Dist=Normal, Score=CRPS())

# # fit the model on the training data
# ngb.fit(X_train, y_train)

# # predict on the test data
# y_pred = ngb.predict(X_test)

# # calculate the root mean squared error (RMSE)
# rmse = mean_squared_error(y_test, y_pred, squared=False)
# print('RMSE:', rmse)


In [133]:
# from catboost import CatBoostRegressor

# # define the model
# cat_model = CatBoostRegressor(iterations=80000, learning_rate=0.1, loss_function='RMSE')
# target_col = 'actual_time'
# X = icf.drop(columns=[target_col])
# y = icf[target_col]
# # fit the model
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=100)

# # make predictions on the test set
# y_pred = cat_model.predict(X_test)

# # calculate RMSE
# rmse = mean_squared_error(y_test, y_pred, squared=False)
# print('RMSE:', rmse)



In [134]:
gt_3 = tf['UID']
gt_9 = cat_model.predict(itf)
gt_9 = pd.DataFrame(gt_9)
gt_9.rename(columns={0: "actual_time"}, inplace=True)
submission_z = pd.concat([gt_3, gt_9], axis=1)
submission_z.reset_index(drop=True)
submission_z.to_csv('submission_g.csv', index=False)

In [135]:
df


,UID,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,od_start_time,...,cutoff_timestamp,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,actual_time
0,lWYYrbNEdf,2018-09-15 02:23:18.656742,thanos::sroute:9d8ed3f7-a8af-4393-ad7a-c57a720...,FTL,trip-153697819865649878,IND832109AAB,Jamshedpur_Central_I_3 (Jharkhand),IND834002AAB,Ranchi_Hub (Jharkhand),2018-09-15 02:23:18.656742,...,2018-09-15 04:45:23,158.807515,778.672716,1228.735753,2.782826,77.988686,20.793472,28.475783,3.292773,100.0
1,iNJcHQlPCm,2018-09-30 15:07:48.971326,thanos::sroute:b9142a73-d68f-46ec-9afc-d145479...,FTL,trip-153832006897106333,IND422011AAD,Nashik_TgrniaRD_I (Maharashtra),IND421302AAG,Bhiwandi_Mankoli_HB (Maharashtra),2018-09-30 22:38:13.903069,...,2018-10-01 05:38:30,81.388109,227.479029,NaN,3.625248,68.517914,18.787231,34.612259,4.277350,61.0
2,U6N0M43qwR,2018-09-23 22:15:46.635975,thanos::sroute:883e99fa-50a3-40e0-a2e2-9b12ed6...,FTL,trip-153774094663572416,IND845305AAA,Raxaul_KairiyaT_D (Bihar),NaN,Muzaffrpur_Bbganj_I (Bihar),2018-09-24 03:56:37.016232,...,2018-09-24 09:21:30,134.486898,400.072069,1188.943158,4.415969,70.475386,24.371978,29.867950,5.322072,69.0
3,RrJUneuYNj,2018-09-21 05:11:44.833176,thanos::sroute:96a80600-40e1-436b-9161-fa68f9e...,FTL,trip-153750670483274503,IND842001AAA,Muzaffrpur_Bbganj_I (Bihar),IND854326AAB,Purnia_Central_H_2 (Bihar),2018-09-22 10:08:32.593022,...,2018-09-22 14:09:17,262.866674,510.123878,999.421258,2.946647,101.876844,19.588977,27.794019,4.265799,300.0
4,7QLdD7iRC9,2018-09-18 01:10:49.936747,thanos::sroute:b43ec86d-59c1-452c-b4a9-807d357...,FTL,trip-153723305402762987,IND411033AAA,Pune_Tathawde_H (Maharashtra),IND501359AAE,Hyderabad_Shamshbd_H (Telangana),2018-09-18 01:10:49.936747,...,2018-09-18 12:14:05,379.389280,NaN,NaN,1.998790,46.571301,22.671495,26.971844,3.553652,286.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72153,dNuV95njhn,2018-09-29 12:35:30.065739,thanos::sroute:ace9e650-93bd-49ab-9400-8ed7a29...,FTL,trip-153822453006548340,IND501359AAE,Hyderabad_Shamshbd_H (Telangana),IND585104AAA,Gulbarga_Nehrugnj_I (Karnataka),2018-09-29 12:35:30.065739,...,2018-09-29 20:08:31,135.001150,NaN,1392.622148,1.950803,55.856356,39.233287,46.999351,1.383605,92.0
72154,ZSrzBfWx77,2018-09-22 11:00:33.571928,thanos::sroute:34b3abb5-f538-4f43-bbeb-5b12c1a...,Carting,trip-153761403357161860,IND562132AAA,Bangalore_Nelmngla_H (Karnataka),IND560083AAB,Bengaluru_Bnnrghta_L (Karnataka),2018-09-22 11:00:33.571928,...,2018-09-22 13:11:21,74.245265,374.255819,NaN,1.927801,35.976514,13.607794,15.139818,1.456469,30.0
72155,XsIYBRVK2V,2018-09-14 23:01:12.630742,thanos::sroute:16dbdb8b-1ca6-4d26-a45c-18d378e...,FTL,trip-153696607263047584,IND629179AAA,Marthandam_Nallur_D (Tamil Nadu),IND627005AAA,Tirunelveli_VdkkuSrt_I (Tamil Nadu),2018-09-15 04:20:06.629681,...,2018-09-15 06:53:24,54.097087,895.612578,902.191591,2.148569,44.101033,23.355458,27.510588,2.346207,74.0
72156,6YwMOShK5E,2018-09-22 06:11:41.998188,thanos::sroute:23372931-5898-4276-970b-0df5070...,FTL,trip-153759670199790692,IND110037AAM,Delhi_Airport_H (Delhi),IND282001AAA,Agra_Idgah_P (Uttar Pradesh),2018-09-22 06:11:41.998188,...,2018-09-22 15:42:27,99.918631,NaN,1393.905281,2.206373,37.771823,18.740087,29.929128,2.499566,122.0


In [136]:
# Define the CatBoost model with updated hyperparameters
# X = icf.drop(columns=[target_col])
# y = icf[target_col]
# # fit the model
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# model = CatBoostRegressor(iterations=100000, 
#                            learning_rate=0.05,
#                            depth=6,
#                            l2_leaf_reg=3,
#                            random_seed=42,
#                            loss_function='RMSE',
#                            eval_metric='RMSE',
#                            use_best_model=True)

# # Train the model
# model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=500)
# from sklearn.metrics import mean_squared_error

# # make predictions on the test set
# y_pred = model.predict(X_test)

# # calculate mean squared error
# mse = mean_squared_error(y_test, y_pred)

# # calculate root mean squared error
# rmse = np.sqrt(mse)

# print("RMSE:", rmse)


In [139]:
from catboost import CatBoostRegressor, Pool

# create train and validation pools
target_col = 'actual_time'
X = icf.drop(columns=[target_col])
y = icf[target_col]
# fit the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_pool = Pool(X_train, y_train)
val_pool = Pool(X_test, y_test)

# define hyperparameters
params = {
    'iterations': 1000,
    'learning_rate': 0.1,
    'eval_metric': 'RMSE',
    'loss_function': 'RMSE',
    'random_seed': 42,
    'od_type': 'Iter',
    'od_wait': 50,
    'boosting_type': 'Ordered'
}

# train the model
model = CatBoostRegressor(**params)
model.fit(train_pool, eval_set=val_pool, verbose=50)

# make predictions
y_pred = model.predict(X_test)

# calculate RMSE
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('RMSE:', rmse)


0:	learn: 545.2864549	test: 544.5388918	best: 544.5388918 (0)	total: 145ms	remaining: 2m 25s
50:	learn: 73.9161919	test: 73.1596193	best: 73.1596193 (50)	total: 4.48s	remaining: 1m 23s
100:	learn: 65.4126240	test: 64.7800227	best: 64.7800227 (100)	total: 8.36s	remaining: 1m 14s
150:	learn: 62.7420654	test: 62.5550187	best: 62.5550187 (150)	total: 11s	remaining: 1m 1s
200:	learn: 59.6778849	test: 60.0284789	best: 60.0284789 (200)	total: 14.8s	remaining: 58.8s
250:	learn: 56.5037167	test: 56.9946296	best: 56.9946101 (249)	total: 19.5s	remaining: 58.2s
300:	learn: 54.9804476	test: 55.4330066	best: 55.4330066 (300)	total: 23.6s	remaining: 54.8s
350:	learn: 54.3641430	test: 54.8122394	best: 54.8122394 (350)	total: 26.8s	remaining: 49.5s
400:	learn: 52.5683964	test: 53.4442648	best: 53.4442648 (400)	total: 29.9s	remaining: 44.7s
450:	learn: 50.8459149	test: 51.9794342	best: 51.9794342 (450)	total: 33.6s	remaining: 40.9s
500:	learn: 49.6086038	test: 50.9638449	best: 50.9638449 (500)	total: 37

In [140]:
from catboost import CatBoostRegressor

# define the model
cat_model = CatBoostRegressor(iterations=120000, learning_rate=0.05, loss_function='RMSE')
target_col = 'actual_time'
X = icf.drop(columns=[target_col])
y = icf[target_col]
# fit the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=100)

# make predictions on the test set
y_pred = cat_model.predict(X_test)

# calculate RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('RMSE:', rmse)


0:	learn: 572.9718610	test: 572.0076729	best: 572.0076729 (0)	total: 18.9ms	remaining: 25m 9s
100:	learn: 72.7705316	test: 72.6821877	best: 72.6821877 (100)	total: 1.55s	remaining: 20m 26s
200:	learn: 60.7628705	test: 60.9059454	best: 60.9059454 (200)	total: 3.05s	remaining: 20m 12s
300:	learn: 54.4335296	test: 55.0975572	best: 55.0975572 (300)	total: 4.61s	remaining: 20m 21s
400:	learn: 49.8725119	test: 51.0657568	best: 51.0657568 (400)	total: 6.99s	remaining: 23m 8s
500:	learn: 46.3758123	test: 48.1026459	best: 48.1026459 (500)	total: 8.53s	remaining: 22m 34s
600:	learn: 43.6939983	test: 46.0496629	best: 46.0496629 (600)	total: 10.1s	remaining: 22m 20s
700:	learn: 41.5484301	test: 44.3337463	best: 44.3337463 (700)	total: 11.7s	remaining: 22m 4s
800:	learn: 39.8284665	test: 43.0166983	best: 43.0166983 (800)	total: 13.2s	remaining: 21m 48s
900:	learn: 38.3377216	test: 41.9197190	best: 41.9197190 (900)	total: 14.8s	remaining: 21m 37s
1000:	learn: 36.9822041	test: 40.8456299	best: 40.845

In [141]:
!pip install optuna
!pip install catboost


In [145]:
import catboost as cb
from sklearn.metrics import mean_squared_error

def objective(trial):
    # Define the hyperparameters to optimize
    params = {
        'iterations': 2000,
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 1e1),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
        'grow_policy': 'Depthwise',
        'eval_metric': 'RMSE',
        'random_seed': 42,
        'task_type': 'GPU',
        'devices': '0'
    }
    from catboost import Pool
    target_col = 'actual_time'
    X = icf.drop(columns=[target_col])
    y = icf[target_col]
# fit the model
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    train_pool = Pool(X_train, y_train)
    valid_pool = Pool(X_test, y_test)

    # Train the model with the current set of hyperparameters
    model = cb.CatBoostRegressor(**params)
    model.fit(train_pool, eval_set=valid_pool, verbose=False)
    
    # Predict the target variable on the validation set and calculate the RMSE
    y_pred = model.predict(valid_pool)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    return rmse


In [146]:
import optuna

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)


[I 2023-04-04 22:43:02,291] A new study created in memory with name: no-name-d9d4086d-21fd-44bf-93b0-dcc9c192c8a0
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  # Remove the CWD from sys.path while we load stuff.
[I 2023-04-04 22:43:46,330] Trial 0 finished with value: 44.87161335583764 and parameters: {'learning_rate': 0.0020374856503605596, 'depth': 9, 'l2_leaf_reg': 0.02559772513867245, 'bagging_temperature': 0.24918666538172718}. Best is trial 0 with value: 44.87161335583764.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` ins

In [147]:
best_params = study.best_params


In [150]:
model = cb.CatBoostRegressor(**best_params)
model.fit(train_pool, eval_set= Pool(X_test, y_test), verbose=False)


In [151]:
y_pred = model.predict(Pool(X_test, y_test))
rmse = mean_squared_error(y_test, y_pred, squared=False)


In [152]:
rmse

40.678998371654224